In [1]:
import os
import numpy as np

from drp_template.default_params import update_parameters_file, check_output_folder
import drp_template.input_output as io

from heidi_toolkit.image import plot_moduli, save_figure

In [18]:
def export_model(filename, data, voxel_size=None, dtype='>f4', paramsfile='parameters.json', order='F'):
    """
    Export model data to a binary file with optional dimension, dtype and endian specification.
    Also updates the parameters file with the voxel size.

    Parameters:
    -----------
    model : ndarray
        The model data to be exported.
    path : str
        The directory path to save the exported data.
    filename : str
        The name of the file being exported.
    voxel_size : float
        The size of the voxel.
    dtype : str, optional
        The data type of the exported data. Can be 'uint8' or 'uint16'. Default is 'uint8'.
    endian : str, optional
        The byte order of the exported data, if dtype is 'uint16'. Can be 'big' or 'little'. Default is 'little'.
    paramsfile : str, optional
        The name of the parameters file to be updated. Default is 'parameters.json'.
    order : str, optional
        The memory layout of the data. Can be 'F' for Fortran ordering and 'C' for C ordering. Default is 'F'.

    Returns:
    --------
    None
    
    Writes moduli data to a binary file in big-endian or little-endian format with Fortran or C ordering.

    Args:
        filename (str): The path to the directory where the binary file will be created.
        data (numpy.ndarray): The moduli data to be written.
        dtype (str, optional): The data type of the elements in the array. Defaults to '>f4'.
        mode (str, optional): The file mode used for opening the binary file.
            - 'w+': Open for reading and writing, create the file if it does not exist.
            - 'wb+': Open for reading and writing in binary mode, create the file if it does not exist.
        order (str, optional): The memory layout of the data.
            - 'F' for Fortran ordering (column-major),
            - 'C' for C ordering (row-major).
            Defaults to 'F'.

    Returns:
        None

    Notes:
        - The 'dtype' parameter specifies the byte order of the data. '>f4' stands for big-endian, and '<f4' stands for little-endian.
        - The 'mode' parameter determines the file opening mode. The 'b' flag in 'wb+' ensures binary mode.
        - The 'order' parameter determines the memory layout of the data, with 'F' for Fortran ordering and 'C' for C ordering.

    Examples:
        To write data in little-endian format with C ordering:
        ```python
        export_endian('output.bin', my_data, dtype='<f4', mode='wb+', order='C')
        ```
    """
    # Create the directory if it doesn't exist
    output_path = check_output_folder()
    
    file_path = os.path.join(output_path, filename)
    
    if dtype == 'uint8':
        flat_data = data.flatten()
        packed_data = struct.pack(f'>{data.size}B', *flat_data)
        with open(file_path + '.raw', 'wb') as f:
            f.write(packed_data)

    elif dtype == 'uint16':
        if endian_check_data == '<' or (endian_check_data == '=' and endian_check_sys == 'little'):
            with open(file_path + '_le.raw', 'wb') as f:
                data.tofile(f, sep='', format='<')
        elif endian_check_data == '>' or (endian_check_data == '=' and endian_check_sys == 'big'):
            with open(file_path + '_be.raw', 'wb') as f:
                data.tofile(f, sep='', format='>')
        else:
            raise ValueError("Invalid endian value!")

    elif dtype in ['>f4', '<f4']:

    
    if order == 'F':
        # Check the shape of the data
        if len(data.shape) == 4:
            nx, ny, nz, dim = np.shape(data)
            data_tmp = np.zeros((nz, nx, ny, dim), dtype=dtype)
            data_tmp[:, :, :, :] = data[:, :, :, :]
        elif len(data.shape) == 3:
            nx, ny, nz = np.shape(data)
            data_tmp = np.zeros((nz, nx, ny), dtype=dtype)
            data_tmp[:, :, :] = data[:, :, :]
        else:
            raise ValueError("Data should be a 3D or 4D array")
    
        data = data_tmp
        model_reshaped = data.reshape(data.size, order=order)
        model_reshaped.T.tofile(file_path)
        print(f"data saved: {os.path.join(output_path, filename)}")
        
    elif order == 'C':
        # Check the shape of the data
        if len(data.shape) == 4:
            nx, ny, nz, dim = np.shape(data)
            data_tmp = np.zeros((nx, ny, nz, dim), dtype=dtype)
            data_tmp[:, :, :, :] = data[:, :, :, :]
        elif len(data.shape) == 3:
            nx, ny, nz = np.shape(data)
            data_tmp = np.zeros((nx, ny, nz), dtype=dtype)
            data_tmp[:, :, :] = data[:, :, :]
        else:
            raise ValueError("Data should be a 3D or 4D array")
    
        data = data_tmp
        model_reshaped = data.reshape(data.size, order=order)
        model_reshaped.T.tofile(os.path.join(output_path, filename + '.raw'))

        print(f"data saved: {os.path.join(output_path, filename + '.raw')}")
        export_header(os.path.join(output_path, filename + '.raw'), data)
    else:
        raise ValueError("Invalid dtype value!")

    # Get the dimensions of the model
    dimensions = {'nx': nx, 'ny': ny, 'nz': nz}

    # Update the parameters file
    update_parameters_file(paramsfile, voxel_size=voxel_size)
    update_parameters_file(paramsfile, dtype=dtype)
    update_parameters_file(paramsfile, order=order)
    update_parameters_file(paramsfile, dimensions=dimensions)

    
def export_header(filename, data):
    # Extract the dimensions from the shape of the input data
    if len(data.shape) == 4:
        n1, n2, n3, n4 = [format(dim, '06') for dim in data.shape]
    elif len(data.shape) == 3:
            n1, n2, n3 = [format(dim, '06') for dim in data.shape]
    else:
        raise ValueError("Data should be a 3D or 4D array")
    
    # Add the conditional statement
    if n3 == '000001':
        n3 = n4
    n4 = '1'

    # Define the dimensions
    dim = [n1, n2, n3, n4]

    # Define the header
    header = "SEPlib Headerfile: quick & dirty: please always check !!!\n"
    header += "so far: only n1,n2,n3,n4 will be part of output\n\n"
    if filename == 'moduli':
        header += f"sets next: in=\"moduli\"\n\n"
    else:
        header += f"sets next: in=\"./{filename}moduli\"\n\n"
    header += f"n1={n1}\nn2={n2}\nn3={n3}\nn4={n4}\n"


    filename = filename+'header'
    # Write the header and dimensions to a new file
    with open(filename, 'w') as f:
        f.write(header)

    return print(f"header saved: {filename}")

In [ ]:
def write_data_to_file(file_path, data, format):
    with open(file_path, 'wb') as f:
        data.tofile(f, sep='', format=format)

def reshape_and_save_data(data, dtype, order, file_path):
    if len(data.shape) not in [3, 4]:
        raise ValueError("Data should be a 3D or 4D array")

    if order == 'F':
        data = np.moveaxis(data, [0, 1, 2], [1, 2, 0])
    elif order != 'C':
        raise ValueError("Invalid order value!")

    model_reshaped = data.reshape(data.size, order=order)
    model_reshaped.T.tofile(file_path)

    print(f"data saved: {file_path}")

def export_model(filename, data, dtype, order, output_path):
    file_path = os.path.join(output_path, filename)
    endian_check_data = dtype[0]
    endian_check_sys = sys.byteorder

    if dtype == 'uint8':
        flat_data = data.flatten()
        packed_data = struct.pack(f'>{data.size}B', *flat_data)
        with open(file_path + '.raw', 'wb') as f:
            f.write(packed_data)
    elif dtype == 'uint16':
        if endian_check_data in ['<', '='] and endian_check_sys == 'little':
            write_data_to_file(file_path + '_le.raw', data, '<')
        elif endian_check_data in ['>', '='] and endian_check_sys == 'big':
            write_data_to_file(file_path + '_be.raw', data, '>')
        else:
            raise ValueError("Invalid endian value!")
    elif dtype in ['>f4', '<f4']:
        reshape_and_save_data(data, dtype, order, file_path)
    else:
        raise ValueError("Invalid dtype value!")

In [4]:
# Set model dimensions 
nz = 600
nx = 600
ny = 600

# Set elastic properties for your model
vp_phase1 = 4000.0
vshear_phase1 = 2000.0
rho_phase1 = 1000.0

In [9]:
# Initialize model array with vacuum values
model = np.zeros((nz, nx, ny, 3), dtype='>f4')

# Fill model with phase 1 values from 2 to nz-2, 2 to nx-2, 2 to ny-2
model[2:nz-2, 2:nx-2, 2:ny-2, 0] = vp_phase1*vp_phase1*rho_phase1
model[2:nz-2, 2:nx-2, 2:ny-2, 1] = vshear_phase1*vshear_phase1*rho_phase1
model[2:nz-2, 2:nx-2, 2:ny-2, 2] = rho_phase1

filename = 'export_moduli'

In [19]:
# Export model as big endian Fortran file
export_model(filename=filename, data=model, dtype='<f4', order='C')

data saved: /Users/martin/Library/Mobile Documents/com~apple~CloudDocs/MYDATA/CODING_WORLD/PYTHON_WORLD/Digital_Rock_Physics_Template/examples/output/export_moduli.raw
header saved: /Users/martin/Library/Mobile Documents/com~apple~CloudDocs/MYDATA/CODING_WORLD/PYTHON_WORLD/Digital_Rock_Physics_Template/examples/output/export_moduli.rawheader


In [17]:
#  Import exported moduli for visualization and correctness check
file_path = f"output/{filename}"
model_in, modelheader_in = io.import_model(file_path,dimension= dtype="<f4", order='C')

Parameters filename: export_moduli.json


ValueError: At least two dimensions (nx, ny, nz) must be provided.

In [ ]:
# Generate the moduli plot
fig, ax = plot_moduli(data=model_in, title=None, im=0, grid_spacing=None, slice=None, aspect_ratio='squared')

# Save the generated plot to a file
save_figure(fig, filename=f"./data/figure_moduli", format=None, dpi=None)